<a href="https://colab.research.google.com/github/hujwantshubham/BRS/blob/main/Copy_of_Untitled30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install openpyxl
streamlit
pandas
openpyxl


# Step 2: Import required libraries
import openpyxl
from openpyxl.styles import PatternFill
from openpyxl import load_workbook
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
from datetime import datetime
import re

# Global variables
uploaded_filename = None
full_bank_txns = None
full_book_txns = None
processed_wbs = {}  # to store the workbook per stage

# Define color fills for conditional formatting.
green_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")  # matched
red_fill   = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")    # unmatched

# =============================================================================
# 1) UPLOAD
# =============================================================================
upload_button = widgets.FileUpload(accept='.xlsx', multiple=False, description="Upload File")
display(upload_button)

def handle_upload(change):
    global uploaded_filename
    if upload_button.value:
        uploaded_filename = list(upload_button.value.keys())[0]
        with open(uploaded_filename, 'wb') as f:
            f.write(upload_button.value[uploaded_filename]['content'])
        print(f"Uploaded File: {uploaded_filename}")

upload_button.observe(handle_upload, names='value')

# =============================================================================
# Utility: Extract numeric tokens. For each token with >=4 digits, also add its last 4 digits.
# =============================================================================
def extract_numbers_with_tail(text):
    if text is None:
        return set()
    tokens = re.findall(r'\d+', str(text))
    result = set(tokens)
    for token in tokens:
        if len(token) >= 4:
            result.add(token[-4:])
    return result

# =============================================================================
# Common Extraction Routine: Returns lists of bank and book transactions.
# =============================================================================
def extract_transactions(ws):
    bank_txns = []
    book_txns = []
    # Data rows start at row 2.
    for row in ws.iter_rows(min_row=2):
        # Bank fields (Columns A–E):
        bank_date = row[0].value         # Column A: Bank Date
        bank_narration = row[2].value      # Column C: Transaction Particulars (narration)
        bank_amount = row[3].value         # Column D: Bank Amount
        bank_dr_cr = row[4].value          # Column E: Bank DR/CR
        # Book fields (Columns F–P):
        book_amount = row[5].value         # Column F: Book Amount
        book_date = row[8].value           # Column I: Book Date
        book_description = row[13].value   # Column N: Book Description (narration)
        book_dr_cr = row[15].value         # Column P: Book DR/CR

        if bank_amount is not None:
            bank_txns.append({
                'row': row,
                'date': bank_date,
                'amount': bank_amount,
                'dr_cr': bank_dr_cr,
                'narration': bank_narration,
                'cell': row[3],          # For coloring the bank Amount cell.
                'matched': False,
                'narration_matched': False
            })
        if book_amount is not None:
            book_txns.append({
                'row': row,
                'date': book_date,
                'amount': book_amount,
                'dr_cr': book_dr_cr,
                'description': book_description,
                'cell': row[5],          # For coloring the book Amount cell.
                'matched': False,
                'reversal': False,
                'narration_matched': False
            })
    return bank_txns, book_txns

# =============================================================================
# Routine: Create Extra Sheets (Reversals, Missing in Book, Extra in Book)
# =============================================================================
def create_extra_sheets(wb, bank_txns, book_txns):
    ws = wb.active
    # Reversal detection among Book transactions.
    reversal_rows = []
    for i in range(len(book_txns)):
        if not book_txns[i]['reversal']:
            for j in range(i+1, len(book_txns)):
                if not book_txns[j]['reversal']:
                    if book_txns[i]['amount'] == book_txns[j]['amount']:
                        if ((book_txns[i]['dr_cr'] == "DR" and book_txns[j]['dr_cr'] == "CR") or
                            (book_txns[i]['dr_cr'] == "CR" and book_txns[j]['dr_cr'] == "DR")):
                            if book_txns[i]['date'] is not None and book_txns[j]['date'] is not None:
                                diff_days = abs((book_txns[i]['date'] - book_txns[j]['date']).days)
                                if diff_days <= 10:
                                    book_txns[i]['reversal'] = True
                                    book_txns[j]['reversal'] = True
                                    reversal_rows.append(book_txns[i]['row'])
                                    reversal_rows.append(book_txns[j]['row'])
                                    break
    rev_sheet = wb.create_sheet("Reversals")
    for r in reversal_rows:
        rev_sheet.append([cell.value for cell in r[5:16]])

    miss_sheet = wb.create_sheet("Missing in Book")
    for txn in bank_txns:
        if not txn['matched']:
            miss_sheet.append([cell.value for cell in txn['row']])

    extra_sheet = wb.create_sheet("Extra in Book")
    for txn in book_txns:
        if not txn['matched']:
            extra_sheet.append([cell.value for cell in txn['row']])

# =============================================================================
# Stage 2: Matched Amounts (Amount Only)
# =============================================================================
def process_amount_only():
    global uploaded_filename
    if not uploaded_filename:
        print("Please upload a file first.")
        return None
    wb = load_workbook(uploaded_filename)
    ws = wb.active
    bank_txns, book_txns = extract_transactions(ws)
    matched_pairs = []
    for btxn in bank_txns:
        for bktxn in book_txns:
            if not btxn['matched'] and not bktxn['matched']:
                if btxn['amount'] == bktxn['amount']:
                    btxn['matched'] = True
                    bktxn['matched'] = True
                    matched_pairs.append((btxn, bktxn))
                    break
    for txn in bank_txns:
        txn['cell'].fill = green_fill if txn['matched'] else red_fill
    for txn in book_txns:
        txn['cell'].fill = green_fill if txn['matched'] else red_fill
    create_extra_sheets(wb, bank_txns, book_txns)
    return wb

# =============================================================================
# Stage 3: DRCR/CRDR (Amount + DR/CR)
# =============================================================================
def process_drcr():
    global uploaded_filename
    if not uploaded_filename:
        print("Please upload a file first.")
        return None
    wb = load_workbook(uploaded_filename)
    ws = wb.active
    bank_txns, book_txns = extract_transactions(ws)
    matched_pairs = []
    for btxn in bank_txns:
        for bktxn in book_txns:
            if not btxn['matched'] and not bktxn['matched']:
                if btxn['amount'] == bktxn['amount'] and (
                    (btxn['dr_cr'] == "DR" and bktxn['dr_cr'] == "CR") or
                    (btxn['dr_cr'] == "CR" and bktxn['dr_cr'] == "DR")
                ):
                    btxn['matched'] = True
                    bktxn['matched'] = True
                    matched_pairs.append((btxn, bktxn))
                    break
    for txn in bank_txns:
        txn['cell'].fill = green_fill if txn['matched'] else red_fill
    for txn in book_txns:
        txn['cell'].fill = green_fill if txn['matched'] else red_fill
    create_extra_sheets(wb, bank_txns, book_txns)
    return wb

# =============================================================================
# Stage 4: Date (Amount + DR/CR + Date)
# =============================================================================
def process_date():
    global uploaded_filename
    if not uploaded_filename:
        print("Please upload a file first.")
        return None
    wb = load_workbook(uploaded_filename)
    ws = wb.active
    bank_txns, book_txns = extract_transactions(ws)
    matched_pairs = []
    for btxn in bank_txns:
        for bktxn in book_txns:
            if not btxn['matched'] and not bktxn['matched']:
                if btxn['amount'] == bktxn['amount'] and (
                    (btxn['dr_cr'] == "DR" and bktxn['dr_cr'] == "CR") or
                    (btxn['dr_cr'] == "CR" and bktxn['dr_cr'] == "DR")
                ):
                    if btxn['date'] is not None and bktxn['date'] is not None:
                        diff_days = abs((btxn['date'] - bktxn['date']).days)
                        if diff_days <= 10:
                            btxn['matched'] = True
                            bktxn['matched'] = True
                            matched_pairs.append((btxn, bktxn))
                            break
                    else:
                        btxn['matched'] = True
                        bktxn['matched'] = True
                        matched_pairs.append((btxn, bktxn))
                        break
    for txn in bank_txns:
        txn['cell'].fill = green_fill if txn['matched'] else red_fill
    for txn in book_txns:
        txn['cell'].fill = green_fill if txn['matched'] else red_fill
    create_extra_sheets(wb, bank_txns, book_txns)
    return wb

# =============================================================================
# Stage 5: Check Narration (All Conditions)
#   Standard matching (Amount + DR/CR + Date) plus narration matching (numeric tokens)
# =============================================================================
def process_full():
    global uploaded_filename, full_bank_txns, full_book_txns
    if not uploaded_filename:
        print("Please upload a file first.")
        return None
    wb = load_workbook(uploaded_filename)
    ws = wb.active
    bank_txns, book_txns = extract_transactions(ws)
    standard_pairs = []
    for btxn in bank_txns:
        for bktxn in book_txns:
            if not btxn['matched'] and not bktxn['matched']:
                if btxn['amount'] == bktxn['amount'] and (
                    (btxn['dr_cr'] == "DR" and bktxn['dr_cr'] == "CR") or
                    (btxn['dr_cr'] == "CR" and bktxn['dr_cr'] == "DR")
                ):
                    if btxn['date'] is not None and bktxn['date'] is not None:
                        diff_days = abs((btxn['date'] - bktxn['date']).days)
                        if diff_days <= 10:
                            btxn['matched'] = True
                            bktxn['matched'] = True
                            standard_pairs.append((btxn, bktxn))
                            break
                    else:
                        btxn['matched'] = True
                        bktxn['matched'] = True
                        standard_pairs.append((btxn, bktxn))
                        break
    for txn in bank_txns:
        txn['cell'].fill = green_fill if txn['matched'] else red_fill
    for txn in book_txns:
        txn['cell'].fill = green_fill if txn['matched'] else red_fill

    # Narration matching: For each standard pair, check numeric tokens.
    final_pairs = []
    for pair in standard_pairs:
        btxn, bktxn = pair
        bank_tokens = extract_numbers_with_tail(btxn.get('narration'))
        book_tokens = extract_numbers_with_tail(bktxn.get('description'))
        if bank_tokens and book_tokens:
            inter = bank_tokens.intersection(book_tokens)
            if inter:
                btxn['narration_matched'] = True
                bktxn['narration_matched'] = True
                final_pairs.append((btxn, bktxn, inter))
    create_extra_sheets(wb, bank_txns, book_txns)
    # Create "Matched Entries" sheet.
    me_sheet = wb.create_sheet("Matched Entries")
    header = next(ws.iter_rows(min_row=1, max_row=1))
    bank_header = [cell.value for cell in header[0:5]]
    book_header = [cell.value for cell in header[5:16]]
    extra_cols = ["Matched Bank Tokens", "Matched Book Tokens"]
    me_sheet.append(bank_header + book_header + extra_cols)
    for pair in final_pairs:
        btxn, bktxn, tokens = pair
        bank_data = [cell.value for cell in btxn['row'][0:5]]
        book_data = [cell.value for cell in bktxn['row'][5:16]]
        token_str = ", ".join(sorted(tokens))
        me_sheet.append(bank_data + book_data + [token_str, token_str])

    # Save these processed lists for use in additional matching.
    full_bank_txns = bank_txns
    full_book_txns = book_txns
    return wb

# =============================================================================
# Stage 6: Additional Matching (Final)
#   - Use only those bank transactions not in "Matched Entries" (narration_matched == False)
#   - Match them with remaining book transactions (each book used only once)
#   - Append remaining unmatched bank entries and also remaining unmatched book entries.
# =============================================================================
def process_additional():
    global full_bank_txns, full_book_txns, uploaded_filename
    if full_bank_txns is None or full_book_txns is None:
        print("Please run the 'Check Narration (All Conditions)' stage first.")
        return None
    # Use stored lists so that flags are preserved.
    remaining_bank = [b for b in full_bank_txns if not b.get('narration_matched')]
    remaining_book = [b for b in full_book_txns if not b.get('narration_matched')]
    additional_matches = []
    used_book_indices = set()
    # For each unmatched bank, find one unmatched book that matches based on amount and DR/CR.
    for btxn in remaining_bank:
        for idx, bktxn in enumerate(remaining_book):
            if idx in used_book_indices:
                continue
            if btxn['amount'] == bktxn['amount'] and (
                (btxn['dr_cr'] == "DR" and bktxn['dr_cr'] == "CR") or
                (btxn['dr_cr'] == "CR" and bktxn['dr_cr'] == "DR")
            ):
                additional_matches.append((btxn, bktxn))
                used_book_indices.add(idx)
                btxn['matched'] = True   # mark as matched in additional stage
                bktxn['matched'] = True
                break
    # Create "Additional Matching" sheet.
    wb = load_workbook(uploaded_filename)  # Reload base workbook to add new sheet.
    ws = wb.active
    add_sheet = wb.create_sheet("Additional Matching")
    header = next(ws.iter_rows(min_row=1, max_row=1))
    bank_header = [cell.value for cell in header[0:5]]
    book_header = [cell.value for cell in header[5:16]]
    add_sheet.append(bank_header + book_header + ["Additional Match Status"])

    # Output the additional matched pairs.
    for pair in additional_matches:
        btxn, bktxn = pair
        bank_data = [cell.value for cell in btxn['row'][0:5]]
        book_data = [cell.value for cell in bktxn['row'][5:16]]
        add_sheet.append(bank_data + book_data + ["Matched"])

    # Append remaining unmatched bank transactions.
    for btxn in remaining_bank:
        if not btxn.get('matched'):
            bank_data = [cell.value for cell in btxn['row'][0:5]]
            blank_book = ["" for _ in range(11)]  # Book columns F-P: 11 columns.
            add_sheet.append(bank_data + blank_book + ["Unmatched Bank"])
            btxn['cell'].fill = red_fill  # color unmatched bank amount red.

    # Append remaining unmatched book transactions.
    for bktxn in remaining_book:
        if not bktxn.get('matched'):
            blank_bank = ["" for _ in range(5)]   # Bank columns A-E: 5 columns.
            book_data = [cell.value for cell in bktxn['row'][5:16]]
            add_sheet.append(blank_bank + book_data + ["Unmatched Book"])
            bktxn['cell'].fill = red_fill

    return wb

# =============================================================================
# Create Processing Buttons in Order:
# (1) Upload is already displayed.
# (2) Matched Amounts (Amount Only)
# (3) DRCR/CRDR (Amount + DR/CR)
# (4) Date (Amount + DR/CR + Date)
# (5) Check Narration (All Conditions)
# (6) Additional Matching (Final)
# =============================================================================
btn_amount = widgets.Button(description="Matched Amounts (Amount Only)", button_style='info')
btn_drcr = widgets.Button(description="DRCR/CRDR (Amount + DR/CR)", button_style='info')
btn_date = widgets.Button(description="Date (Amount + DR/CR + Date)", button_style='info')
btn_narration = widgets.Button(description="Check Narration (All Conditions)", button_style='success')
btn_additional = widgets.Button(description="Additional Matching (Final)", button_style='warning')
display(btn_amount, btn_drcr, btn_date, btn_narration, btn_additional)

def start_amount(b):
    wb = process_amount_only()
    if wb:
        processed_wbs["amount"] = wb
        print("Processed: Matched Amounts (Amount Only).")

def start_drcr(b):
    wb = process_drcr()
    if wb:
        processed_wbs["drcr"] = wb
        print("Processed: DRCR/CRDR (Amount + DR/CR).")

def start_date(b):
    wb = process_date()
    if wb:
        processed_wbs["date"] = wb
        print("Processed: Date (Amount + DR/CR + Date).")

def start_narration(b):
    wb = process_full()
    if wb:
        processed_wbs["full"] = wb
        print("Processed: Check Narration (All Conditions).")

def start_additional(b):
    wb = process_additional()
    if wb:
        processed_wbs["additional"] = wb
        print("Processed: Additional Matching (Final).")

btn_amount.on_click(start_amount)
btn_drcr.on_click(start_drcr)
btn_date.on_click(start_date)
btn_narration.on_click(start_narration)
btn_additional.on_click(start_additional)

# =============================================================================
# Final Step: Download Button with dropdown selection.
# =============================================================================
download_options = widgets.Dropdown(
    options=[("Matched Amounts", "amount"),
             ("DRCR/CRDR", "drcr"),
             ("Date", "date"),
             ("Check Narration", "full"),
             ("Additional Matching", "additional")],
    description="Download Stage:"
)
btn_download = widgets.Button(description="Download Processed File", button_style='danger')
display(download_options, btn_download)

def download_processed(b):
    key = download_options.value
    if key in processed_wbs and processed_wbs[key]:
        filename = f"Processed_BRS_{key}.xlsx"
        processed_wbs[key].save(filename)
        files.download(filename)
    else:
        print("Please run the processing stage first.")

btn_download.on_click(download_processed)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00


FileUpload(value={}, accept='.xlsx', description='Upload File')

Button(button_style='info', description='Matched Amounts (Amount Only)', style=ButtonStyle())

Button(button_style='info', description='DRCR/CRDR (Amount + DR/CR)', style=ButtonStyle())

Button(button_style='info', description='Date (Amount + DR/CR + Date)', style=ButtonStyle())

Button(button_style='success', description='Check Narration (All Conditions)', style=ButtonStyle())

Button(button_style='warning', description='Additional Matching (Final)', style=ButtonStyle())

Dropdown(description='Download Stage:', options=(('Matched Amounts', 'amount'), ('DRCR/CRDR', 'drcr'), ('Date'…

Button(button_style='danger', description='Download Processed File', style=ButtonStyle())